# Using GLM Models to finish QA using a search API and re-ranking.

**This tutorial is available in English and is attached below the Chinese explanation**

本脚本参考了[OpenAI CookBook](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_a_search_API.ipynb)的代码，并使用 GLM 系列模型实现同样的功能。

**步骤 1：搜索**

1. 用户提问。
2. 由 GLM-4 生成潜在查询列表。
3. 并行执行搜索查询。

**步骤 2：重新排序**

1. 使用每个结果的嵌入来计算与生成的用户问题理想答案的语义相似度。
2. 基于这个相似度度量对结果进行排序和筛选。

**步骤 3：回答**

1. 给出最顶端的搜索结果，模型生成用户问题的答案，包括引用和链接。

这种混合方法提供了相对较低的延迟，并且可以集成到任何现有的搜索端点中，而不需要维护一个向量数据库！我们将使用[News API](https://newsapi.org/)作为搜索领域的示例。

This script refers to the code in [OpenAI CookBook](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_a_search_API.ipynb) and implements the same functionality using the GLM family of models.

**Step 1: Search**

1. User asks a question.
2. Generate a list of potential queries by GLM-4.
3. Execute search queries in parallel.

**Step 2: Rerank**

1. Use the embedding of each result to calculate the semantic similarity with the generated ideal answer to the user's question.
2. Sort and filter the results based on this similarity measure.

**Step 3: Answer**

1. Given the top search results, the model generates an answer to the user's question, including citations and links.

This hybrid approach provides relatively low latency and can be integrated into any existing search endpoint without the need to maintain a vector database! We will use the [News API](https://newsapi.org/) as an example in the search domain.

In [1]:
import json
from numpy import dot
import requests
from tqdm.notebook import tqdm
from openai import OpenAI

client = OpenAI(
    api_key="your ZhipuAI API keys",
    base_url="https://open.bigmodel.cn/api/paas/v4/"
)

def json_glm(input: str):
    completion = client.chat.completions.create(
        model="glm-4-0520",
        messages=[
            {"role": "system", "content": "请你严格按照用户指令的要求执行，必须按照 JSON BLOB的格式输出"},
            {"role": "user", "content": input},
        ],
        temperature=0.5
    )

    text = completion.choices[0].message.content
    json_content = text.strip().strip('```json').strip('```').strip()
    parsed = json.loads(json_content)
    return parsed


/Users/zr/Code/glm-cookbook/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. 确定搜索的问题

我们需要确定本次搜索的问题

We need to determine the problem of this search

In [2]:
USER_QUESTION = "谁赢得了 NBA 总冠军？谁是 MVP？"

现在，为了尽可能详尽，我们使用该模型根据这个问题生成一个多样化的查询列表。

Now, in order to be as exhaustive as possible, we use the model to generate a list of diverse queries based on this question.

In [3]:
QUERIES_INPUT = f"""
您可以访问返回最新新闻文章的搜索 API。生成与此问题相关的搜索查询数组。使用相关关键字的变体进行查询，尽量尽可能通用。包括您能想到的尽可能多的查询，包括和排除术语。这是搜索的关键词例子：
['keyword_1 keyword_2', 'keyword_1', 'keyword_2'] 
发挥创意。您包含的查询越多，找到相关结果的可能性就越大。最好使用英语进行搜索，因为这个网站需要使用英语的关键词。
用户问题：{USER_QUESTION}
返回格式：
{{"queries": ["query_1", "query_2", "query_3"]}}
请你严格按照返回格式的要求输出。
"""

queries = json_glm(QUERIES_INPUT)["queries"]
queries.append(USER_QUESTION)
queries

['NBA champions',
 'NBA championship winner',
 'NBA Finals MVP',
 'NBA Finals winner',
 'NBA championship 2023',
 'NBA Finals MVP 2023',
 'latest NBA champions',
 'who won the NBA Finals',
 'NBA Finals winner 2023',
 'NBA MVP',
 'NBA Finals most valuable player',
 '谁赢得了 NBA 总冠军？谁是 MVP？']

## 2. Search

接着，我们运行搜索。

let's run the searches.

In [4]:
def search_news(
        query: str,
        news_api_key: str,
        num_articles: int = 50,
        from_datetime: str = "2024-06-25",
        to_datetime: str = "2024-06-26",
) -> dict:
    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": news_api_key,
            "pageSize": num_articles,
            "sortBy": "relevancy",
            "from": from_datetime,
            "to": to_datetime,
        },
    )

    return response.json()


articles = []

for query in tqdm(queries):
    result = search_news(query=query, news_api_key="your newsapi.org key")
    if result["status"] == "ok":
        articles = articles + result["articles"]
    else:
        raise Exception(result["message"])

# remove duplicates
articles = list({article["url"]: article for article in articles}.values())

print("Total number of articles:", len(articles))
print("Top 5 articles of query 1:", "\n")

for article in articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print()


  0%|          | 0/12 [00:00<?, ?it/s]

Total number of articles: 16
Top 5 articles of query 1: 

Title: Panthers Thrill NHL Fans in Game 7 Win vs. McDavid, Oilers to Clinch 1st Stanley Cup
Description: You can finally exhale, Florida Panthers fans. Florida not only avoided blowing a 3-0 lead to Connor McDavid and the Edmonton Oilers with Monday's 2-1 victory…
Content: Peter Joneleit/Icon Sportswire via Getty Images
You can finally exhale, Florida Panthers fans.
Flo...

Title: Jason Tatum and Matthew Tkachuk were high school friends, now they’re both champions
Description: The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrate.
Content: The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrat...

Title: NBA playoff parity gives Dunleavy confidence in Warriors' title return
Description: Mike Dunleavy is confident the Warriors will have a chance to compete for a championship next season after watching the 2024 NBA playoffs.
Content: NBA 

我们可以看到，搜索查询通常会返回大量结果，其中许多结果与用户提出的原始问题无关。为了提高最终答案的质量，我们使用嵌入来重新排序和过滤结果。

As we can see, oftentimes, the search queries will return a large number of results, many of which are not relevant to the original question asked by the user. In order to improve the quality of the final answer, we use embeddings to re-rank and filter the results.

## 3. Re-rank

我们首先生成一个假设的理想答案，然后重新排序并与结果进行比较。这有助于优先考虑看起来像好答案的结果，而不是与我们的问题相似的结果。这是我们用来生成假设答案的提示。

We first generate a hypothetical ideal answer to rerank our compare our results against. This helps prioritize results that look like good answers, rather than those similar to our question. Here’s the prompt we use to generate our hypothetical answer.


In [5]:
HA_INPUT = f"""
为用户的问题生成一个假设答案。此答案将用于对搜索结果进行排名。假装您拥有回答所需的所有信息，但不要使用任何实际事实。相反，使用占位符例如 NAME 做了某事，或 NAME 在 PLACE 说了某事，请你用英语来输出。
用户问题: {USER_QUESTION}
格式要求: {{"hypotheticalAnswer": "hypothetical answer text"}}
"""
hypothetical_answer = json_glm(HA_INPUT)["hypotheticalAnswer"]
hypothetical_answer

'TEAM won the NBA Championship, and PLAYER was named MVP.'

现在，让我们为搜索结果和假设答案生成嵌入。然后我们计算这些嵌入之间的余弦距离，从而得到一个语义相似度指标。请注意，我们可以简单地计算点积，而不必进行完整的余弦相似度计算，因为 OpenAI 嵌入在我们的 API 中是经过归一化的。

Now, let's generate embeddings for the search results and the hypothetical answer. We then calculate the cosine distance between these embeddings, giving us a semantic similarity metric. Note that we can simply calculate the dot product in lieu of doing a full cosine similarity calculation since the OpenAI embeddings are returned normalized in our API.


In [6]:
def embeddings(input: list[str]) -> list[list[str]]:
    response = client.embeddings.create(model="embedding-2", input=input)
    return [data.embedding for data in response.data]


hypothetical_answer_embedding = embeddings(hypothetical_answer)[0]
articles_list = [f"{article['title']} {article['description']} {article['content'][0:100]}" for article in articles]
articles_list[0:10]

["Panthers Thrill NHL Fans in Game 7 Win vs. McDavid, Oilers to Clinch 1st Stanley Cup You can finally exhale, Florida Panthers fans. Florida not only avoided blowing a 3-0 lead to Connor McDavid and the Edmonton Oilers with Monday's 2-1 victory… Peter Joneleit/Icon Sportswire via Getty Images\r\nYou can finally exhale, Florida Panthers fans.\r\nFlo",
 'Jason Tatum and Matthew Tkachuk were high school friends, now they’re both champions The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrate. The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrat',
 "NBA playoff parity gives Dunleavy confidence in Warriors' title return Mike Dunleavy is confident the Warriors will have a chance to compete for a championship next season after watching the 2024 NBA playoffs. NBA playoff parity gives Dunleavy confidence in Warriors' title return originally appeared on NBC Sp",
 'Red Sox honor Celtics at Fenway Park

In [7]:
article_embeddings = embeddings(articles_list)

# Calculate cosine similarity
cosine_similarities = []
for article_embedding in article_embeddings:
    cosine_similarities.append(dot(hypothetical_answer_embedding, article_embedding))
cosine_similarities[0:10]

[0.4157394961088908,
 0.5116303590995759,
 0.44501031185827594,
 0.469290430808307,
 0.446827633695725,
 0.5052495840107389,
 0.41411032641360235,
 0.42188560268752007,
 0.4470155901597918,
 0.47972604502479255]

## 4. Using the similarity scores to re-rank the results

最后，我们使用这些相似度分数对结果进行排序和筛选。

Finally, we use these similarity scores to sort and filter the results.


In [8]:
scored_articles = zip(articles, cosine_similarities)

# Sort articles by cosine similarity
sorted_articles = sorted(scored_articles, key=lambda x: x[1], reverse=True)

# Print top 5 articles
print("Top 5 articles:", "\n")

for article, score in sorted_articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print("Score:", score)
    print()


Top 5 articles: 

Title: Jason Tatum and Matthew Tkachuk were high school friends, now they’re both champions
Description: The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrate.
Content: The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrat...
Score: 0.5116303590995759

Title: Jayson Tatum and Matthew Tkachuk were high school friends, now they’re both champions
Description: The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrate.
Content: The Celtics and the Panthers winning the NBA and NHL titles means these BFFs have PLENTY to celebrat...
Score: 0.5052495840107389

Title: Game 7 etc. open thread: someone will make the worst kind of history
Description: The Florida Panthers had a 3-0 lead in the Stanley Cup finals, and then this appeared: The Panthers have been outscored 17-5 since this whatever you call stuff that appears on Truth Social appeare

## 5. Get the answer

最后，我们给出最顶端的搜索结果，模型生成用户问题的答案，包括引用和链接。

Finally, given the top search results, the model generates an answer to the user's question, including citations and links.


In [9]:
formatted_top_results = [
    {
        "title": article["title"],
        "description": article["description"],
        "url": article["url"],
    }
    for article, _score in sorted_articles[0:5]
]

ANSWER_INPUT = f"""
根据给定的搜索结果生成用户问题的答案。
TOP_RESULTS：{formatted_top_results}
USER_QUESTION：{USER_QUESTION}
在答案中包含尽可能多的信息。将相关搜索结果网址引用为 markdown 链接。用中文输出答案。
"""

completion = client.chat.completions.create(
    model="glm-4-0520",
    messages=[{"role": "user", "content": ANSWER_INPUT}],
    temperature=0.5,
    stream=False,
)
completion.choices[0].message.content

'根据搜索结果，NBA 总冠军是由波士顿凯尔特人队赢得的。不过，搜索结果中并没有提到谁是本次NBA总决赛的MVP。\n\n更多信息可以查看以下链接：[Jason Tatum 和 Matthew Tkachuk 是高中好友，现在他们都是冠军](https://www.sbnation.com/2024/6/24/24185383/jason-tatum-matthew-tkachuk-celtics-panthers-nba-nhl-champions) 和 [Jayson Tatum 和 Matthew Tkachuk 是高中好友，现在他们都是冠军](https://www.sbnation.com/2024/6/24/24185383/jayson-tatum-matthew-tkachuk-celtics-panthers-nba-nhl-champions)。\n\n如果您需要MVP的具体信息，可能需要进一步查找相关资料。'